In [1]:
from marubatsu import Marubatsu
from ai import ai_by_score

def ai5s(mb, debug=False):
    def eval_func(mb):
        my_turn = Marubatsu.CROSS if mb.turn == Marubatsu.CIRCLE else Marubatsu.CIRCLE
        if mb.status == my_turn:
            return 1
        else:
            return 0

    return ai_by_score(mb, eval_func, debug=debug)

In [2]:
from ai import ai_match, ai2, ai5, ai6, ai7

ai_match(ai=[ai5s, ai5])

ai5s VS ai5
count     win    lose    draw
o        6815    2770     415
x        2699    6846     455
total    9514    9616     870

ratio     win    lose    draw
o       68.2%   27.7%    4.2%
x       27.0%   68.5%    4.5%
total   47.6%   48.1%    4.3%



In [3]:
def ai5s(mb, debug=False):
    def eval_func(mb):
        if mb.status == Marubatsu.CIRCLE or mb.status == Marubatsu.CROSS:
            return 1
        else:
            return 0

    return ai_by_score(mb, eval_func, debug=debug)

In [4]:
ai_match(ai=[ai5s, ai5])

ai5s VS ai5
count     win    lose    draw
o        6841    2736     423
x        2677    6879     444
total    9518    9615     867

ratio     win    lose    draw
o       68.4%   27.4%    4.2%
x       26.8%   68.8%    4.4%
total   47.6%   48.1%    4.3%



In [5]:
def restart(self):
    self.initialize_board()
    self.turn = Marubatsu.CIRCLE     
    self.move_count = 0
    self.status = Marubatsu.PLAYING
    self.last_move = -1, -1 
    self.last_turn = None

Marubatsu.restart = restart

In [6]:
def move(self, x, y):
    if self.place_mark(x, y, self.turn):
        self.last_turn = self.turn
        self.turn = Marubatsu.CROSS if self.turn == Marubatsu.CIRCLE else Marubatsu.CIRCLE  
        self.move_count += 1
        self.status = self.judge()
        self.last_move = x, y

Marubatsu.move = move

In [7]:
def ai5s(mb, debug=False):
    def eval_func(mb):
        if mb.status == mb.last_turn:
            return 1
        else:
            return 0

    return ai_by_score(mb, eval_func, debug=debug)

In [8]:
ai_match(ai=[ai5s, ai5])

ai5s VS ai5
count     win    lose    draw
o        6885    2673     442
x        2698    6842     460
total    9583    9515     902

ratio     win    lose    draw
o       68.8%   26.7%    4.4%
x       27.0%   68.4%    4.6%
total   47.9%   47.6%    4.5%



In [9]:
ai_match(ai=[ai5, ai5])

ai5 VS ai5
count     win    lose    draw
o        6958    2595     447
x        2642    6928     430
total    9600    9523     877

ratio     win    lose    draw
o       69.6%   25.9%    4.5%
x       26.4%   69.3%    4.3%
total   48.0%   47.6%    4.4%



In [10]:
ai_match(ai=[ai5s, ai5s])

ai5s VS ai5s
count     win    lose    draw
o        6864    2726     410
x        2681    6898     421
total    9545    9624     831

ratio     win    lose    draw
o       68.6%   27.3%    4.1%
x       26.8%   69.0%    4.2%
total   47.7%   48.1%    4.2%



In [11]:
mb = Marubatsu()

mb.move(1, 1)
mb.move(1, 0)
mb.move(0, 0)
mb.move(2, 0)
mb.move(0, 1)
mb.move(1, 2)
print(mb)

print("ai5")
for i in range(10):
    print(ai5(mb))
    
print("ai5s")
for i in range(10):
    print(ai5s(mb))

Turn o
oxx
oo.
.X.

ai5
(2, 1)
(2, 1)
(2, 1)
(2, 1)
(2, 1)
(2, 1)
(2, 1)
(2, 1)
(2, 1)
(2, 1)
ai5s
(2, 2)
(2, 2)
(2, 2)
(0, 2)
(2, 1)
(2, 1)
(2, 2)
(0, 2)
(2, 2)
(2, 1)


In [12]:
from collections import defaultdict

def count_marks(self, coord, dx, dy):
    x, y = coord   
    count = defaultdict(int)
    for _ in range(self.BOARD_SIZE):
        count[self.board[x][y]] += 1
        x += dx
        y += dy

    return count

Marubatsu.count_marks = count_marks

In [13]:
mb = Marubatsu()

print(mb)
print(mb.count_marks((0, 0), 1, 0))

mb.move(0, 0)
print(mb)
print(mb.count_marks((0, 0), 1, 0))

mb.move(1, 0)
print(mb)
print(mb.count_marks((0, 0), 1, 0))

mb.move(2, 0)
print(mb)
print(mb.count_marks((0, 0), 1, 0))

Turn o
...
...
...

defaultdict(<class 'int'>, {'.': 3})
Turn x
O..
...
...

defaultdict(<class 'int'>, {'o': 1, '.': 2})
Turn o
oX.
...
...

defaultdict(<class 'int'>, {'o': 1, 'x': 1, '.': 1})
Turn x
oxO
...
...

defaultdict(<class 'int'>, {'o': 2, 'x': 1})


In [14]:
def ai6s(mb, debug=False):
    def eval_func(mb):
        # 自分が勝利している場合は、評価値として 1 を返す
        if mb.status == mb.last_turn:
            return 1

        # 相手の手番で相手が勝利できる場合は評価値として -1 を返す
        # 横方向と縦方向の判定
        for i in range(mb.BOARD_SIZE):
            count = mb.count_marks(coord=[0, i], dx=1, dy=0)
            if count[mb.turn] == 2 and count[Marubatsu.EMPTY] == 1:
                return -1
            count = mb.count_marks(coord=[i, 0], dx=0, dy=1)
            if count[mb.turn] == 2 and count[Marubatsu.EMPTY] == 1:
                return -1
        # 左上から右下方向の判定
        count = mb.count_marks(coord=[0, 0], dx=1, dy=1)
        if count[mb.turn] == 2 and count[Marubatsu.EMPTY] == 1:
            return -1
        # 右上から左下方向の判定
        count = mb.count_marks(coord=[2, 0], dx=-1, dy=1)
        if count[mb.turn] == 2 and count[Marubatsu.EMPTY] == 1:
            return -1

        # それ以外の場合は評価値として 0 を返す
        return 0

    return ai_by_score(mb, eval_func, debug=debug)        

In [15]:
ai_match(ai=[ai6s, ai6])

ai6s VS ai6
count     win    lose    draw
o        3161    1672    5167
x        1723    3114    5163
total    4884    4786   10330

ratio     win    lose    draw
o       31.6%   16.7%   51.7%
x       17.2%   31.1%   51.6%
total   24.4%   23.9%   51.6%



In [16]:
mb = Marubatsu()

mb.move(1, 1)
mb.move(1, 0)
mb.move(0, 0)
mb.move(2, 0)
mb.move(0, 1)
print(mb)

print("ai6")
for i in range(10):
    print(ai6(mb))
    
print("ai6s")
for i in range(10):
    print(ai6s(mb))

Turn x
oxx
Oo.
...

ai6
(2, 1)
(2, 1)
(2, 1)
(2, 1)
(2, 1)
(2, 1)
(2, 1)
(2, 1)
(2, 1)
(2, 1)
ai6s
(2, 1)
(1, 2)
(1, 2)
(0, 2)
(1, 2)
(0, 2)
(0, 2)
(0, 2)
(2, 2)
(2, 2)


In [17]:
ai_match(ai=[ai6s, ai2])

ai6s VS ai2
count     win    lose    draw
o        8858     195     947
x        6937     912    2151
total   15795    1107    3098

ratio     win    lose    draw
o       88.6%    1.9%    9.5%
x       69.4%    9.1%   21.5%
total   79.0%    5.5%   15.5%



In [18]:
def ai7s(mb, debug=False):
    def eval_func(mb):
        # 真ん中のマスに着手している場合は、評価値として 2 を返す
        if mb.last_move == (1, 1):
            return 2
    
        # 自分が勝利している場合は、評価値として 1 を返す
        if mb.status == mb.last_turn:
            return 1

        # 相手の手番で相手が勝利できる場合は評価値として -1 を返す
        # 横方向と縦方向の判定
        for i in range(mb.BOARD_SIZE):
            count = mb.count_marks(coord=[0, i], dx=1, dy=0)
            if count[mb.turn] == 2 and count[Marubatsu.EMPTY] == 1:
                return -1
            count = mb.count_marks(coord=[i, 0], dx=0, dy=1)
            if count[mb.turn] == 2 and count[Marubatsu.EMPTY] == 1:
                return -1
        # 左上から右下方向の判定
        count = mb.count_marks(coord=[0, 0], dx=1, dy=1)
        if count[mb.turn] == 2 and count[Marubatsu.EMPTY] == 1:
            return -1
        # 右上から左下方向の判定
        count = mb.count_marks(coord=[2, 0], dx=-1, dy=1)
        if count[mb.turn] == 2 and count[Marubatsu.EMPTY] == 1:
            return -1

        # それ以外の場合は評価値として 0 を返す
        return 0

    return ai_by_score(mb, eval_func, debug=debug)  

In [19]:
ai_match(ai=[ai7s, ai7])

ai7s VS ai7
count     win    lose    draw
o        2950     416    6634
x         417    2910    6673
total    3367    3326   13307

ratio     win    lose    draw
o       29.5%    4.2%   66.3%
x        4.2%   29.1%   66.7%
total   16.8%   16.6%   66.5%

